In [1]:
import paddle
import numpy as np
import paddle.dataset.mnist as mnist
import paddle.fluid as fluid
from PIL import Image
paddle.enable_static()

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


In [2]:
dropout_layer=paddle.nn.Dropout(p=0.2)
def convolutional_neural_network(input):
    conv1=fluid.layers.conv2d(input=input,num_filters=32,filter_size=(3,3),stride=1)
    pool1=fluid.layers.pool2d(input=conv1,pool_size=2,pool_stride=1,pool_type='max')
    conv2=fluid.layers.conv2d(input=pool1,num_filters=64,filter_size=(3,3),stride=1)
    pool2=fluid.layers.pool2d(input=conv2,pool_size=2,pool_stride=1,pool_type='max')
    pool2=dropout_layer(pool2)
    fc=fluid.layers.fc(input=pool2,size=10,act='softmax')
    return fc

image=fluid.data(name='image',shape=[None,1,28,28],dtype='float32')
model=convolutional_neural_network(image)
infer_program=fluid.default_main_program().clone(for_test=True)

这里只是将卷积神经网络进行了封装，拆开来当然可以,这里用了paddle.nn
因为在paddle后续的版本中将移除fluid，paddle.nn将在后续动态图模式中细讲
在未来的使用中动态图模式才是主流

In [3]:
label=fluid.data(name='lable',shape=[None,1],dtype='int64')
cost=fluid.layers.cross_entropy(input=model,label=label)
avg_cost=fluid.layers.mean(cost)
acc=fluid.layers.accuracy(input=model,label=label)
test_program=fluid.default_main_program().clone(for_test=True)

In [4]:
optimizer=fluid.optimizer.AdamOptimizer(learning_rate=0.001)
opts=optimizer.minimize(avg_cost)

使用了adam梯度下降算法

In [5]:
place=fluid.CUDAPlace(0)
exe=fluid.Executor(place)
exe.run(fluid.default_startup_program())

[]

In [6]:
train_reader=paddle.batch(reader=mnist.train(),batch_size=128)
test_reader=paddle.batch(reader=mnist.test(),batch_size=128)
feeder=fluid.DataFeeder(place=place,feed_list=[image,label])

本代码主要展示卷积神经网络的使用，关于如何采用自定义数据集将在后续实例中展示

In [7]:
for pass_id in range(2):
    for batch_id,data in enumerate(train_reader()):
        train_cost,train_acc=exe.run(program=fluid.default_main_program(),
            feed=feeder.feed(data),
            fetch_list=[avg_cost, acc])
        if batch_id%100==0:
            print('Pass:%d, Batch"%d, Cost:%0.5f, Accuracy:%0.5f'
                    %(pass_id, batch_id, train_cost[0], train_acc[0]))
    test_accs=[]
    test_costs=[]
    for batch_id,data in enumerate(test_reader()):
        test_cost,test_acc=exe.run(program=test_program,
        feed=feeder.feed(data),
        fetch_list=[avg_cost,acc])
        test_accs.append(test_acc[0])
        test_costs.append(test_cost[0])
    test_costs=(sum(test_costs))/len(test_costs)
    test_acc=(sum(test_accs))/len(test_accs)
    print('id:%d, Cost:%0.5f, Accuracy:%0.5f' %(pass_id,test_cost,test_acc))

Pass:0, Batch"0, Cost:6.81210, Accuracy:0.10156
Pass:0, Batch"100, Cost:0.29407, Accuracy:0.91406
Pass:0, Batch"200, Cost:0.16173, Accuracy:0.96875
Pass:0, Batch"300, Cost:0.19109, Accuracy:0.96094
Pass:0, Batch"400, Cost:0.19079, Accuracy:0.93750
id:0, Cost:0.01452, Accuracy:0.96875
Pass:1, Batch"0, Cost:0.13655, Accuracy:0.96875
Pass:1, Batch"100, Cost:0.08126, Accuracy:0.96875
Pass:1, Batch"200, Cost:0.06314, Accuracy:0.97656
Pass:1, Batch"300, Cost:0.12812, Accuracy:0.97656
Pass:1, Batch"400, Cost:0.21127, Accuracy:0.95312
id:1, Cost:0.65440, Accuracy:0.95332
